In [ ]:
import pandas as pd

import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import warnings
warnings.filterwarnings('ignore')
     

df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/30/p1.csv',encoding= 'cp949')
df.head()

,Age,Gender,BMI,DBP,FPG,Chol,Tri,HDL,LDL,ALT,CCR,FFPG
0,26,1,20.1,81,5.80,4.36,0.86,0.90,2.43,12.0,63.8,5.40
1,40,1,17.7,54,4.60,3.70,1.02,1.50,2.04,9.2,70.3,4.10
2,40,2,19.7,53,5.30,5.87,1.29,1.75,3.37,10.1,61.1,4.85
3,43,1,23.1,71,4.50,4.05,0.74,1.27,2.60,36.5,73.4,5.30
4,36,1,26.5,82,5.54,6.69,3.49,0.91,3.64,69.3,67.5,5.53


In [ ]:
df['Gender'] = df['Gender'].astype(str)

In [ ]:
fig, axes = plt.subplots(6,2 ,figsize = (7,10))

cols = df.columns

for i, ax in enumerate(axes.flat[:]) : 
    col_i = df.columns[i]
    if col_i == 'Gender' :
        target = df[col_i].value_counts()
        sns.barplot(x = target.index, y = target.values, ax = ax) 
        
    else : 
        sns.histplot(x= df[col_i], data = df, ax = ax)
        
        
plt.show()

<module 'matplotlib.pyplot' from '/lib/python3.12/site-packages/matplotlib/pyplot.py'>

In [ ]:
# #이상치 제거

def remove_outlier(df) :
    num_cols = df.select_dtypes(exclude = 'object').columns
    
    iqr_dic = {}
    for col in num_cols:
        iqr = np.quantile(df[col], 0.75) - np.quantile(df[col], 0.25)
        upper = np.quantile(df[col], 0.75) + 1.5 * iqr 
        lower = np.quantile(df[col], 0.25) - 1.5 * iqr 
        
        df = df[(df[col] <= upper) & (df[col] >= lower)]
        return df 
    
remove_df = remove_outlier(df)


fig, axes = plt.subplots(1,2, figsize = (10,7))
sns.boxplot(data = df, ax = axes[0])
sns.boxplot(data=remove_df, ax = axes[1])

<AxesSubplot:>

train test set을 DBP컬럼 기준으로 7:3 비율로 나누고 잘 나뉘었는지 통계적으로 나타내라

In [ ]:
r_df = remove_df.copy()

from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind

X = r_df.drop(columns =['DBP'], axis =1)
y = r_df['DBP']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3 , random_state = 1234)

s, p = ttest_ind(y_train, y_test)
# 두 집단의 평균이 같다
print(p)

0.8973956290599971


독립변수의 차원축소의 필요성을 논하고, 필요에 따라 차원을 축소하고 불필요하다면 그 근거를 논하시오

In [ ]:
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler

# #차원 축소 검증을 위한 Gender변수 전환
r_df['Gender'] = r_df['Gender'].astype(int)

pca_df = r_df.copy()

pca_df = pca_df.dropna().drop(columns = ['DBP'], axis = 1)

ss = MinMaxScaler()
scaled_df = pd.DataFrame(ss.fit_transform(pca_df) , columns = pca_df.columns)

pca = PCA()
pca.fit(scaled_df)


pc_table = pd.DataFrame({'ratio' : pca.explained_variance_ , 
              'cumsum' : pca.explained_variance_.cumsum() })
              
pc_table.index = ['PC_' + str(x) for x in pc_table.index ]
print(pc_table.T)

PC_0      PC_1      PC_2      PC_3      PC_4      PC_5      PC_6  \
ratio   0.230754  0.075124  0.039121  0.017778  0.016125  0.009862  0.002549   
cumsum  0.230754  0.305878  0.344999  0.362777  0.378903  0.388765  0.391314   

            PC_7      PC_8      PC_9     PC_10  
ratio   0.002189  0.001943  0.001869  0.000996  
cumsum  0.393503  0.395446  0.397315  0.398311


In [ ]:
# #다중 공선성 확인
from patsy import dmatrices 
from statsmodels.stats.outliers_influence import variance_inflation_factor 

r_df = r_df.dropna()

vif_df = pd.DataFrame()
vif_df['factor'] = r_df.columns 
vif_df['vif'] = [variance_inflation_factor(r_df.values , i ) for i in range(r_df.shape[1])] 
vif_df.T

,0,1,2,3,4,5,6,7,8,9,10,11
factor,Age,Gender,BMI,DBP,FPG,Chol,Tri,HDL,LDL,ALT,CCR,FFPG
vif,15.700888,11.332832,62.96735,49.99728,59.596111,78.65768,3.330166,8.942627,55.575387,3.004014,22.360487,24.298843


작업 후 데이터가 회귀분석의 기본가정 따르는지 설명

In [ ]:
from statsmodels.formula.api import ols
from scipy.stats import shapiro
x_cols= ''
for col in r_df.columns : 
    x_cols += col + "+"
    
combined_col = x_cols[:-1]

model = ols('DBP~' + combined_col , data = r_df).fit()
pred = model.predict(r_df) 

resid = model.resid
fitted = model.fittedvalues
# 잔차는 정규분포이다. - 정규성
_, p = shapiro(resid)
print(p)


# 선형성 : 독립변수와 종속변수간의 선형적인 관계가 있어야 한다. (산점도) 잔차화 Fitted
plt.scatter(x = fitted, y = resid)
plt.show()

# 독립성 : 관측치들의 간의 상관관계가 없어야 한다. 다중공선성을 확인/ 더빈왓슨 검정
from statsmodels.stats.stattools import durbin_watson 
durbin = durbin_watson(resid)
print(durbin)

1.754723263975184e-26
0.33516402560759095


<module 'matplotlib.pyplot' from '/lib/python3.12/site-packages/matplotlib/pyplot.py'>

 회귀분석 알고리즘 3개를 선택하고 선정이유와 장단점 비교

In [ ]:
# 다중공선성의 문제가 있는 데이터를 규제가 있는 회귀식으로 과대 적합을 방지한다.
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import ttest_ind
from sklearn.pipeline import Pipeline

X = r_df.drop(columns =['DBP'], axis =1)
y = r_df['DBP']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3 , random_state = 1234)


# Lasso
def l_model (x_train, y_train) :
    la = Lasso()
    
    param_grid = {'lasso__alpha' : np.logspace(-8 , 8, 100) } 
    
    pipeline = Pipeline([('scaler', StandardScaler()) , ('lasso', la)])
    
    grid = GridSearchCV(pipeline, param_grid, cv= 4)
    grid.fit(x_train, y_train)
    return grid

lasso = l_model(x_train, y_train)    
la_pred = lasso.predict(x_test)

def r_model (x_train, y_train) :
    
    ridge = Ridge()
    
    param = {'ridge__alpha' : np.logspace(-8, 8, 100) } 
    
    pipeline = Pipeline([ ('scaler' , StandardScaler()) , ('ridge',ridge) ] )
    grid = GridSearchCV(pipeline, param , cv = 3)
    
    grid.fit(x_train, y_train)
    return grid
    
ridge = r_model(x_train,y_train)
r_pred = ridge.predict(x_test)


def ela_model (x_train, y_train) :
    
    ela = ElasticNet()
    
    param = {'ElasticNet__alpha' : np.logspace(-8, 8, 100) } 
    
    pipeline = Pipeline([ ('scaler' , StandardScaler()) , ('ElasticNet', ela) ] )
    grid = GridSearchCV(pipeline, param , cv = 3)
    
    grid.fit(x_train, y_train)
    return grid
    
ela = ela_model(x_train,y_train)
ela_pred = ela.predict(x_test)

# 평가
print(np.sqrt(mean_squared_error(la_pred, y_test)) ) 
print(np.sqrt(mean_squared_error(r_pred, y_test)) )
print(np.sqrt(mean_squared_error(ela_pred, y_test)) )

Error: 

3-3 3-2에서 가장 성능 좋은 알고리즘을 이용하여 K-Fold 교차검증을 수행하시오.

In [ ]:
from sklearn.model_selection import KFold , cross_val_score

kfold = KFold(n_splits = 5, shuffle = True, random_state = 1234)

ela = ElasticNet()
score = cross_val_score(ela, X , y, cv = kfold)
print(np.mean(score))

0.13617442418289744


In [ ]:
def ela_model (X, y) :
    ela = ElasticNet()
    kfold = KFold(n_splits = 5)
    
    cv_result = []
    
    for train_idx, test_idx in kfold.split(X) : 
        x_train, x_test = X.iloc[train_idx] , X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx] , y.iloc[test_idx] 
        
        param = {'ElasticNet__alpha' : np.logspace(-8, 8, 100) } 
        
        pipeline = Pipeline([ ('scaler' , StandardScaler()) , ('ElasticNet', ela) ] )
        grid = GridSearchCV(pipeline, param , cv = 3)
        
        grid.fit(x_train, y_train)
        pred = grid.predict(x_test) 
        
        rmse = mean_squared_error(pred, y_test) 
        cv_result.append(rmse)
    return cv_result
    
    
result = ela_model(X, y)
print(result)

Error: 

4-1 발생시각을 통해 평일인지 주말인지를 구분하는 '주말여부' 범주형 변수 추가하고 데이터 분포를 확인하라. (월 ~ 금은 평일, 토요일과 일요일을 주말)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/30/p2_v2.csv')
df.head(3)

,사고년도,가해자성별,가해자연령,가해차종,가해자신체상해정도,피해자성별,피해자연령,피해차종,피해자신체상해정도,사고유형,가해자법규위반,기상상태,시각,사건번호
0,2015,남,46세,승합차,상해없음,여,64세,자전거,경상,차대차 - 기타,안전운전불이행,맑음,2015-07-03_10시,ID_796
1,2019,남,53세,승용차,상해없음,여,57세,자전거,경상,차대차 - 후진중충돌,안전운전불이행,맑음,2019-10-12_10시,ID_6697
2,2015,남,39세,승용차,상해없음,남,63세,자전거,중상,차대차 - 기타,기타,맑음,2015-03-23_20시,ID_919


In [ ]:
df['가해자신체상해정도'].value_counts()

가해자신체상해정도
상해없음    7290
기타불명     228
경상       196
부상신고     112
Name: count, dtype: int64

In [ ]:
# 시각 전처리
df['시각'] = pd.to_datetime(df['시각'] , format = '%Y-%m-%d_%H시')

# 주말여부
df['주말여부'] = df['시각'].dt.day_name().map(lambda x : '주말' if x in ['Saturday' ,'Sunday'] else '평일')

target = df['주말여부'].value_counts()
print(target)
plt.bar(x = target.index, height = target.values)
plt.show()

주말여부
평일    5950
주말    1876
Name: count, dtype: int64


<module 'matplotlib.pyplot' from '/lib/python3.12/site-packages/matplotlib/pyplot.py'>

 '피해자신체상해정도' 에 따라 각 독립변수들이 유의한지 통계적 검정하라.

In [ ]:
df1 = df[(df['가해자연령'] != '불명') & (df['피해자연령'] != '불명') & (df['가해자신체상해정도'] != '기타불명') ]

df1.loc[:,'가해자연령'] = df1['가해자연령'].str.split('세').str[0].astype(int)
df1.loc[:, '피해자연령'] = df1['피해자연령'].str.split('세').str[0].astype(int) 

# 1,2 개
# df1['피해자신체상해정도'].value_counts()
from scipy.stats import ttest_ind , chi2_contingency 

num_col = ['가해자연령' ,'피해자연령']
cat_col = ['가해자성별' , '가해자법규위반','사고유형','가해차종' ,'가해자신체상해정도','피해차종','기상상태','주말여부']

lst_1 = []
for col in num_col :
    target = df1[[col,'피해자신체상해정도']]
    
    group1 = target[target['피해자신체상해정도'] == '경상'][col]
    group2 = target[target['피해자신체상해정도'] == '중상'][col]
    
    s , p_val = ttest_ind(list(group1), list(group2)) 
    lst_1.append([col, p_val])
    
print(lst_1) 


lst_2 = []
for col in cat_col :
    cross = pd.crosstab(df1[col] ,df1['피해자신체상해정도']) 
    c_stat, p_val, df, ex = chi2_contingency(cross)
    lst_2.append([col, p_val])
    
print(lst_2)

[['가해자연령', 0.8240408329165454], ['피해자연령', 3.536123158387766e-78]]
[['가해자성별', 0.41602485883817075], ['가해자법규위반', 0.010085497789013008], ['사고유형', 0.014714233112966935], ['가해차종', 7.504530948461324e-05], ['가해자신체상해정도', 0.040448856155605256], ['피해차종', 0.0011310413757866793], ['기상상태', 0.3602291654306536], ['주말여부', 0.8110975919725951]]


In [ ]:
final_df = pd.concat([
pd.DataFrame(lst_1 , columns = ['Col', 'P_value']),
pd.DataFrame(lst_2 , columns = ['Col', 'P_value'])],
axis = 0
)
# 유의하다
# 유의하지 않다.
final_df['Check'] = final_df['P_value'].map(lambda x : '유의' if x < 0.05 else '유의하지않다' )
print(final_df)

Col       P_value   Check
0      가해자연령  8.240408e-01  유의하지않다
1      피해자연령  3.536123e-78      유의
0      가해자성별  4.160249e-01  유의하지않다
1    가해자법규위반  1.008550e-02      유의
2       사고유형  1.471423e-02      유의
3       가해차종  7.504531e-05      유의
4  가해자신체상해정도  4.044886e-02      유의
5       피해차종  1.131041e-03      유의
6       기상상태  3.602292e-01  유의하지않다
7       주말여부  8.110976e-01  유의하지않다


에서 유의한 변수들만을 가지고 '피해자신체상해정도'를 기준으로 SMOTE 오버샘플링을 수행하고 샘플링 데이터와 기존데이터를 합친 전체 데이터에 대해 범주형변수는 변수별 빈도를 나타내고 연속형이면 평균을 나타내시오

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder


final_cols = final_df[final_df['Check'] == '유의']['Col'].to_list()
final_cols.append('피해자신체상해정도')

# Label Encoding 
for col in final_cols :
    if df1[col].dtypes == 'object' :
        le = LabelEncoder()
        df1[col] = le.fit_transform(df1[col])
        
        
x = df1[final_cols]
y = df1['피해자신체상해정도']


smote = SMOTE(random_state =123)
s_x , s_y =smote.fit_resample(x, y)

all_x = pd.concat([x, s_x], axis =0)
all_x.head(3)

for col in all_x.columns :
    if col == '피해자연령' :
        print(all_x[col].mean()) 
        
    else : 
        print(all_x[col].value_counts())

46.106577052328376
가해자법규위반
9     10713
7      2645
3      1139
1       608
2       425
8       310
10      297
11      250
5       192
6        47
4        20
12       14
0         4
Name: count, dtype: int64
사고유형
5     4755
11    4612
10    3842
6      954
0      599
4      505
9      501
2      338
7      305
3       87
1       69
8       50
12      47
Name: count, dtype: int64
가해차종
5     10454
9      2868
11     1528
8       768
6       678
7       247
1        53
10       41
0        13
2         6
4         5
3         3
Name: count, dtype: int64
가해자신체상해정도
2    15971
0      426
1      267
Name: count, dtype: int64
피해차종
6    14731
2     1593
5      226
3       60
4       48
7        2
1        2
0        2
Name: count, dtype: int64
피해자신체상해정도
0    9066
1    7598
Name: count, dtype: int64


데이터를 가지고 '피해자신체상해정도' 을 종속변수로 하여 로지스틱회귀분석, XGB 분류 분류 모델을 만들고 성능 비교를 하고 영향력 있는 변수를 확인하라

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, accuracy_score
import xgboost

x_train, x_test, y_train, y_test = train_test_split(s_x, s_y, test_size =0.3, random_state = 1244) 

lr = LogisticRegression()
lr.fit(x_train, y_train)
lr_pred = lr.predict(x_test)

lr_f1_score = accuracy_score(lr_pred, y_test)

xgb = XGBClassifier()
xgb.fit(x_train, y_train)
xgb_pred = xgb.predict(x_test)

xgb_f1_score = accuracy_score(y_test, xgb_pred)

xgboost.plot_importance(xgb)

<AxesSubplot:>

6.1 연령대별로 헤드셋 선호도에 차이가 있는지를 검정하기 위한 귀무가설과 연구가설을 설정해라.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/adp/30/p6.csv') 
df['나이'].value_counts()
df.head(3)

,나이,선호도
0,청년,2
1,장년,2
2,청년,2


In [ ]:
from scipy.stats import chi2_contingency 

cross = pd.crosstab(df['나이'], df['선호도'])

stat, p_val , df, expected = chi2_contingency(cross)

아이를 6명 가진 가정이 5 가족이고, 아들/딸을 출생할 확률은 0.5

In [ ]:
from scipy.stats import binom
# 아이의 확률 - 가족의 확률을 구해야함 
#딸이 4명 이상인 가구가 3 가족 이상일 확률을 구하라. (소숫점 이하 4번째 자리에서 반올림)
rv = binom(6, 0.5)

#딸이 4명 이상일 확률 = p 
daughter_4 = 1- rv.cdf(3)
print(daughter_4)
# 3가족 이상일 확률
print(1 - binom.cdf(2 , 5 , daughter_4) ) 

#  5가족 중 몇 가족이 4명 이상 딸을 가질 것으로 기대할 수 있는가?
# 기대값 = 가구 수 * 딸 

print(5 * daughter_4)

0.34375
0.2255459427833557
1.71875
